In [1]:
import torch
import torch.nn as nn
from core.datasets import fetch_dataloader
from lescroartdata import fetch_dataloader_lesc
import core.utils.sync_batchnorm as sync_batchnorm
from core.eisen import EISEN
from core.raft import EvalRAFT
from types import SimpleNamespace
from collections import OrderedDict

/home/galdegh/.conda/envs/eisen/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = SimpleNamespace(
        dataset='playroom',
        batch_size=8,
        num_workers=8,
        compute_flow=True,
        precompute_flow=False,
        flow_threshold=0.5
    )

In [3]:
device = torch.device("cuda:1")

In [4]:
val_loader = fetch_dataloader(args)
model = EISEN()
raft_model = EvalRAFT(flow_threshold=args.flow_threshold)
raft_model.to(device)

AttributeError: 'EvalRAFT' object has no attribute 'to'

In [5]:
type(raft_model)

core.raft.EvalRAFT

In [5]:
state_dict = torch.load('pretrained/tdw_playroom_128x128_ckpt.pth', map_location=device)

In [6]:
new_state_dict = OrderedDict()

for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

In [7]:
model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

EISEN(
  (backbone): ResNet_Deeplab(
    (decoder): ModuleDict(
      (res2): ModuleDict(
        (project_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (fuse_conv): Sequential(
          (0): Conv2d(160, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (res3): ModuleDict(
        (project_conv): Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (fuse_conv): Sequential(
          (0): Conv2d(320, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (res5): ModuleDict(
        (project_conv): ASPP(
          (convs): ModuleList(
            (0): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): Conv2d(2048, 256, kernel_size=(3, 3), stri

In [8]:
data_dict = next(iter(val_loader))

In [9]:
data_dict['img1'].shape

torch.Size([8, 3, 512, 512])

In [10]:
with torch.no_grad():
       _, _, segment_target = raft_model(data_dict['img1'], data_dict['img2'])

/home/galdegh/.conda/envs/eisen/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/galdegh/.conda/envs/eisen/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/galdegh/.conda/envs/eisen/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/galdegh/EISEN/./RAFT/core/raft.py", line 89, in forward
    image2 = 2 * (image2 / 255.0) - 1.0
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 31.74 GiB total capacity; 102.29 MiB already allocated; 8.12 MiB free; 120.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
